In [1]:
from typing_extensions import Self
from dataclasses import dataclass
import math

In [2]:
# Load the input
with open("day5_input.txt") as f:
    day5_input = f.read()

In [3]:
# RangedMapper class
@dataclass
class RangedMapper:
    source_start: int
    destination_start: int
    range_length: int

    @classmethod
    def Parse(cls: Self, map_range_str: str) -> Self:
        values = map_range_str.split()
        assert(len(values)==3)
        return RangedMapper(source_start=int(values[1]),destination_start=int(values[0]),range_length=int(values[2]))

    def maps_value(self: Self, input_value: int) -> bool:
        offset = input_value - self.source_start
        return offset >= 0 and offset < self.range_length

    def apply(self: Self, input_value: int) -> int:
        # Check valid inputs
        assert(self.maps_value(input_value))

        return self.destination_start + input_value - self.source_start


In [4]:
# Quick tests for RangedMapper parsing
map_range_parse_tests = [
    {
        "parse_input": "50 98 2",
        "test_inputs":      [ 98, 99 ],
        "expected_outputs": [ 50, 51 ],
    },
    {
        "parse_input": "52 50 48",
        "test_inputs":      [ 50, 51, 75, 90 ],
        "expected_outputs": [ 52, 53, 77, 92 ],
    }
]

for test_set in map_range_parse_tests:
    test_map = RangedMapper.Parse(test_set["parse_input"])
    for i, test_input in enumerate(test_set["test_inputs"]):
        output = test_map.apply(test_input)
        expected_output = test_set["expected_outputs"][i]
        pass_str = "PASS" if expected_output == output else "FAIL"
        print(f"{pass_str}. Expected: {expected_output}. Actual: {output}.")

PASS. Expected: 50. Actual: 50.
PASS. Expected: 51. Actual: 51.
PASS. Expected: 52. Actual: 52.
PASS. Expected: 53. Actual: 53.
PASS. Expected: 77. Actual: 77.
PASS. Expected: 92. Actual: 92.


In [5]:
# Mapping class
@dataclass
class Mapping:
    name: str
    mappers: list[RangedMapper]

    def __post_init__(self):
        # Sort by source_start
        self.mappers.sort(key=lambda x: x.source_start)
        # Add mappers for self-mapping numbers
        self_mappers = []
        next_unmapped_value = 0
        for mapper in self.mappers:
            # If this mapper doesn't start where the previous one left off, make a self-mapping
            # to fill the gap
            if mapper.source_start != next_unmapped_value:
                self_map_start = next_unmapped_value
                self_map_len = mapper.source_start - self_map_start
                self_mappers.append(RangedMapper(self_map_start,self_map_start,self_map_len))

            # Then we account for this mapper, and the next unmapped value we haven't checked is
            # this mappers source_start plus its range_length
            next_unmapped_value = mapper.source_start

        # Add one last self-mapper from the last unmapped value to infinity
        self_mappers.append(RangedMapper(next_unmapped_value,next_unmapped_value,math.inf))

        # Add the self mappers to the mappers, then sort again
        [self.mappers.append(sm) for sm in self_mappers]
        self.mappers.sort(key=lambda x: x.source_start)

    @classmethod
    def ParseName(cls: Self, map_name_str: str) -> str:
        return map_name_str.split()[0]
    
    # Parse from a block of text consisting of a name line followed by multiple mapping lines
    @classmethod
    def ParseFromBlock(cls: Self, mapping_block_str: str) -> Self:
        lines = mapping_block_str.split("\n")
        name = cls.ParseName(lines[0])
        mappers = [RangedMapper.Parse(line) for line in lines[1:]]
        return Mapping(name=name, mappers=mappers)
    
    def apply(self: Self, input_value: int):
        # Because we have sorted the mappers already, we can just go through the list of mappers
        # until we either find one that maps the input value OR has a source_start above the input
        # value, meaning we have no mapper defined for the value and it should just be returned as-is
        for mapper in self.mappers:
            if mapper.maps_value(input_value):
                return mapper.apply(input_value)
            elif mapper.source_start > input_value:
                return input_value

        return input_value


In [6]:
# Tests for mapping the whole seed-to-soil map
seed_to_soil_example_block = """
seed-to-soil map:
50 98 2
52 50 48
""".strip()

seed_to_soil_parse_test_inputs           = [ 0, 5, 10, 45, 50, 55, 90, 95, 99, 100, 999]
seed_to_soil_parse_test_expected_outputs = [ 0, 5, 10, 45, 52, 57, 92, 97, 51, 100, 999]

seed_to_soil_test_mapping = Mapping.ParseFromBlock(seed_to_soil_example_block)

# This no longer works with the self-mappers added
# # Check that the input parser on the 2nd line was correctly sorted
# output = seed_to_soil_test_mapping.mappers[0].source_start
# expected_output = 50
# pass_str = "PASS" if expected_output == output else "FAIL"
# print(f"Mapper sorting: {pass_str}. Expected: {expected_output}. Actual: {output}.")

for i in range(len(seed_to_soil_parse_test_inputs)):
    output = seed_to_soil_test_mapping.apply(seed_to_soil_parse_test_inputs[i])
    expected_output = seed_to_soil_parse_test_expected_outputs[i]
    pass_str = "PASS" if expected_output == output else "FAIL"
    print(f"{pass_str}. Expected: {expected_output}. Actual: {output}.")

PASS. Expected: 0. Actual: 0.
PASS. Expected: 5. Actual: 5.
PASS. Expected: 10. Actual: 10.
PASS. Expected: 45. Actual: 45.
PASS. Expected: 52. Actual: 52.
PASS. Expected: 57. Actual: 57.
PASS. Expected: 92. Actual: 92.
PASS. Expected: 97. Actual: 97.
PASS. Expected: 51. Actual: 51.
PASS. Expected: 100. Actual: 100.
PASS. Expected: 999. Actual: 999.


In [7]:
def get_seeds_and_mappings_from_input(input_str: str) -> tuple[list[int], dict[str,Mapping]]:
    mappings = {}
    for i, block in enumerate(input_str.split("\n\n")):
        if i == 0: # Seed numbers
            seeds = [int(x) for x in block.split()[1:]]
        else:
            # Mapping blocks
            mapping = Mapping.ParseFromBlock(block)
            mappings[mapping.name] = mapping
    
    return [seeds, mappings]

In [8]:
# Example data
example_input = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

In [9]:
# Part one step-by-step tests
part_one_tests_step_by_step = {
    "seed-to-soil": {
        79: 81,
        14: 14,
        55: 57,
        13: 13,
    },
    "soil-to-fertilizer": {
        81: 81,
        14: 53,
        57: 57,
        13: 52,
    },
    "fertilizer-to-water": {
        81: 81,
        53: 49,
        57: 53,
        52: 41,
    },
    "water-to-light": {
        81: 74,
        49: 42,
        53: 46,
        41: 34,
    },
    "light-to-temperature": {
        74: 78,
        42: 42,
        46: 82,
        34: 34,
    },
    "temperature-to-humidity": {
        78: 78,
        42: 43,
        82: 82,
        34: 35,
    },
    "humidity-to-location": {
        78: 82,
        43: 43,
        82: 86,
        35: 35,
    },
}

example_seeds, example_mappings = get_seeds_and_mappings_from_input(example_input)

for mapping_name, tests in part_one_tests_step_by_step.items():
    mapper = example_mappings[mapping_name]
    for test_input, expected_output in tests.items():
        output = mapper.apply(test_input)
        pass_str = "PASS" if expected_output == output else "FAIL"
        print(f"{mapping_name}: {pass_str}. Expected: {expected_output}. Actual: {output}.")


seed-to-soil: PASS. Expected: 81. Actual: 81.
seed-to-soil: PASS. Expected: 14. Actual: 14.
seed-to-soil: PASS. Expected: 57. Actual: 57.
seed-to-soil: PASS. Expected: 13. Actual: 13.
soil-to-fertilizer: PASS. Expected: 81. Actual: 81.
soil-to-fertilizer: PASS. Expected: 53. Actual: 53.
soil-to-fertilizer: PASS. Expected: 57. Actual: 57.
soil-to-fertilizer: PASS. Expected: 52. Actual: 52.
fertilizer-to-water: PASS. Expected: 81. Actual: 81.
fertilizer-to-water: PASS. Expected: 49. Actual: 49.
fertilizer-to-water: PASS. Expected: 53. Actual: 53.
fertilizer-to-water: PASS. Expected: 41. Actual: 41.
water-to-light: PASS. Expected: 74. Actual: 74.
water-to-light: PASS. Expected: 42. Actual: 42.
water-to-light: PASS. Expected: 46. Actual: 46.
water-to-light: PASS. Expected: 34. Actual: 34.
light-to-temperature: PASS. Expected: 78. Actual: 78.
light-to-temperature: PASS. Expected: 42. Actual: 42.
light-to-temperature: PASS. Expected: 82. Actual: 82.
light-to-temperature: PASS. Expected: 34. 

In [10]:
# Could probably do this a bit smarter...
mapping_order = [
    "seed-to-soil",
    "soil-to-fertilizer",
    "fertilizer-to-water",
    "water-to-light",
    "light-to-temperature",
    "temperature-to-humidity",
    "humidity-to-location",
]

def get_seed_locations(seeds: list[int], mappings: dict[str, Mapping]) -> list[int]:
    # Check that we have the required mappings
    for required_mapping in mapping_order:
        assert(required_mapping in mappings)

    # Find the location for every seed in the input
    locations = []
    for seed in seeds:
        value = seed
        for mapping_name in mapping_order:
            mapping = mappings[mapping_name]
            value = mapping.apply(value)
        locations.append(value)

    return locations

In [11]:
# Part one tests end-to-end
part_one_tests_end_to_end_expected_locations = [82,43,86,35]

example_seeds, example_mappings = get_seeds_and_mappings_from_input(example_input)

example_locations = get_seed_locations(seeds=example_seeds,mappings=example_mappings)

for i, expected_output in enumerate(part_one_tests_end_to_end_expected_locations):
    output = example_locations[i]
    pass_str = "PASS" if expected_output == output else "FAIL"
    print(f"{pass_str}. Expected: {expected_output}. Actual: {output}.")

print(f"Minimum location in example: {min(example_locations)}")


PASS. Expected: 82. Actual: 82.
PASS. Expected: 43. Actual: 43.
PASS. Expected: 86. Actual: 86.
PASS. Expected: 35. Actual: 35.
Minimum location in example: 35


In [ ]:
# Run on the actual input
seeds, mappings = get_seeds_and_mappings_from_input(day5_input)

locations = get_seed_locations(seeds=seeds,mappings=mappings)

print(f"Answer (part one): {min(locations)}")